## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-11-10-51-43 +0000,bbc,Nigeria can't take Venezuelan deportees from U...,https://www.bbc.com/news/articles/c23gz78v5jjo
1,2025-07-11-10-48-34 +0000,bbc,Kurdish PKK burns guns in big step towards end...,https://www.bbc.com/news/articles/cyvj5v67ly9o
2,2025-07-11-10-40-14 +0000,nyt,Trump’s Seesawing on Tariffs Gives the World W...,https://www.nytimes.com/2025/07/11/world/europ...
3,2025-07-11-10-36-37 +0000,nyt,Behind Trump’s Decision to Tax Brazil to Save ...,https://www.nytimes.com/2025/07/11/world/ameri...
4,2025-07-11-10-11-21 +0000,bbc,Everything you need to know about Wafcon 2024 ...,https://www.bbc.com/sport/football/articles/cq...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2336/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
13,trump,11
116,uk,6
22,brazil,6
48,immigration,5
15,tariffs,4


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
65,2025-07-10-12-04-11 +0000,nyt,"Trump Pledges 50% Tariffs Against Brazil, Citi...",https://www.nytimes.com/2025/07/09/world/ameri...,34
40,2025-07-10-20-22-02 +0000,nyt,US-Brazil Tariffs: What to Know About Trump’s ...,https://www.nytimes.com/2025/07/10/world/ameri...,25
54,2025-07-10-15-59-01 +0000,nyt,Trump Fuels Fear Among Immigrant Farm Workers ...,https://www.nytimes.com/2025/07/10/us/politics...,25
41,2025-07-10-20-10-28 +0000,nyt,"Targeting Brazil, Trump Tests Legal Limit of H...",https://www.nytimes.com/2025/07/10/us/politics...,25
68,2025-07-10-11-30-07 +0000,bbc,Brazil vows to match US tariffs after Trump th...,https://www.bbc.com/news/articles/c62dvyv60z7o,25


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
65,34,2025-07-10-12-04-11 +0000,nyt,"Trump Pledges 50% Tariffs Against Brazil, Citi...",https://www.nytimes.com/2025/07/09/world/ameri...
18,19,2025-07-11-08-06-08 +0000,bbc,UK-France migrant deal 'robust' against legal ...,https://www.bbc.com/news/articles/cx24d70gw41o
36,17,2025-07-10-21-38-24 +0000,latimes,Immigration raid at cannabis farm leads to vio...,https://www.latimes.com/california/story/2025-...
9,17,2025-07-11-10-00-00 +0000,latimes,"Arellano: 'La migra, la migra': Inside Hunting...",https://www.latimes.com/california/story/2025-...
33,16,2025-07-10-21-57-54 +0000,latimes,"Cal State L.A. allows online classes, excused ...",https://www.latimes.com/california/story/2025-...
28,16,2025-07-10-23-44-36 +0000,latimes,Los Angeles County sheriff's deputy pleads gui...,https://www.latimes.com/california/story/2025-...
20,12,2025-07-11-05-00-27 +0000,bbc,Starmer and Macron plead for patience in an im...,https://www.bbc.com/news/articles/clynxqdr157o
11,11,2025-07-11-10-00-00 +0000,latimes,Organizers hope the 2028 Summer Olympics prese...,https://www.latimes.com/california/story/2025-...
26,11,2025-07-11-00-52-29 +0000,latimes,New Trump rule immediately bans undocumented i...,https://www.latimes.com/california/story/2025-...
30,11,2025-07-10-22-45-11 +0000,latimes,"As L.A. bakes, duration of heat waves is accel...",https://www.latimes.com/california/story/2025-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
